## Contributor: Yuxin Zhang
## The file is targeted at 
### 1. training the SVR base model for both the order and the congestion model, the desired result is to produce the best parameters for the SVR from the cross validation approach
### 2. the training_set, validatating set is at ratio : 0.64: 0.16

## The Order Model

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

In [3]:
dataset = pd.read_csv("merged_order_dummy.csv")
dataset.head()
dataset.shape


(17555, 128)

In [3]:
dataset[['order_in_section','speed','precipitation','tempurature']] = StandardScaler().fit_transform(dataset[['order_in_section','speed','precipitation','tempurature']])

dataset.head()




order_in_section     speed  precipitation  tempurature  day_of_week_0  \
0         -0.465421  0.349601      -0.397227    -0.538502              0   
1         -0.465421  0.349601      -0.397227     1.081261              0   
2         -0.465421  0.349601      -0.397227     0.618471              0   
3         -0.465421 -0.272144      -0.397227    -0.075712              0   
4         -0.465421 -0.893889      -0.397227     2.006839              0   

   day_of_week_1  day_of_week_2  day_of_week_3  day_of_week_4  day_of_week_5  \
0              0              0              0              1              0   
1              0              0              0              1              0   
2              0              0              0              1              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

    ...     hour_14  hour_15  hour_16  hour_17  hour_18  hour_19  hour_20  \
0   ...           0        0        0        0        0        0        0   
1   ...           0        0        0        0        1        0        0   
2   ...           0        0        0        0        0        1        0   
3   ...           0        0        0        0        0        0        0   
4   ...           0        0        1        0        0        0        0   

   hour_21  hour_22  hour_23  
0        0        0        0  
1        0        0        0  
2        0        0        0  
3        0        0        0  
4        0        0        0  

[5 rows x 128 columns]

In [7]:
import numpy as np
y = np.array(dataset["order_in_section"].values)
X = np.array(dataset.loc[:, "speed"::].values)
print(y,X)

[-0.46542074 -0.46542074 -0.46542074 ... -0.2242002  -0.38501389
 -0.14379336] [[ 0.34960077 -0.39722711 -0.53850151 ...  0.          0.
   0.        ]
 [ 0.34960077 -0.39722711  1.08126053 ...  0.          0.
   0.        ]
 [ 0.34960077 -0.39722711  0.61847138 ...  0.          0.
   0.        ]
 ...
 [ 0.34960077 -0.33289104 -0.76989609 ...  0.          1.
   0.        ]
 [ 0.34960077 -0.39722711 -1.00129067 ...  0.          0.
   1.        ]
 [ 0.34960077 -0.39722711 -1.00129067 ...  0.          0.
   1.        ]]


### directly train the data--validation setup

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10 )
X_train, X_validate, y_train,y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state = 10)


In [11]:
clf = SVR(degree = 1,C = 0.8860298953772168,epsilon = 0.05377575926266184)
clf.fit(X_train, y_train)

SVR(C=0.8860298953772168, cache_size=200, coef0=0.0, degree=1,
  epsilon=0.05377575926266184, gamma='auto', kernel='rbf', max_iter=-1,
  shrinking=True, tol=0.001, verbose=False)

### get the train and validation mse

In [13]:
print(len(y_train),len(y_validate))

11235 2809


In [17]:
from sklearn.metrics import mean_squared_error
y_train_predicted = clf.predict(X_train)
train_mse = mean_squared_error(y_train, y_train_predicted)
print("order_training_mse = %s "%train_mse)

order_training_mse = 0.419602996178578 


In [20]:
y_validate_predicted = clf.predict(X_validate)
validate_mse = mean_squared_error(y_validate, y_validate_predicted)
print("order_validate_mse = %s "%validate_mse)

order_validate_mse = 0.420984487277943 


### training_validation_test split

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10 )
# X_train, X_validate, y_train,y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state=10 )
print("length of training %s || length of test %s"%(len(X_train),len(X_test)))

length of training 14044 || length of test 3511


(14044, 1) (14044, 127)


### do the randomsearch cross validation

In [14]:
## tune C, epsilon, and the kernel parameters.

In [21]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint 
from scipy.stats import uniform as sp_uniform
from sklearn.svm import SVR

# model = SVR(kernel='rbf', gamma='auto_deprecated', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=800, verbose=False, max_iter=1000)
model = SVR()
param_dist = {
    'degree': sp_randint(1,10),
    'C' : sp_uniform(0,2),
    'epsilon':sp_uniform(0.05,0.15)
    
    
}
random_search = RandomizedSearchCV(
    model,
    param_distributions = param_dist,
    cv           = 5,
    n_iter       = 50,  
    random_state = 10 
    
)

In [ ]:
random_search.fit(X_train, y_train)

In [ ]:
print(random_search.best_params_)

### Loss function

In [ ]:
#ε–insensitive

In [16]:
clf = SVR(degree = 1,C = 0.8860298953772168,epsilon = 0.05377575926266184)
clf.fit(X_train, y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=0.8860298953772168, cache_size=200, coef0=0.0, degree=1,
  epsilon=0.05377575926266184, gamma='auto', kernel='rbf', max_iter=-1,
  shrinking=True, tol=0.001, verbose=False)

In [ ]:
from sklearn.metrics import mean_squared_error


In [ ]:
y_predicted = clf.predict(X_tr)

In [17]:
clf.score(X_train, y_train)

0.6302962099276468

## the congestion model

In [ ]:
import pandas as pd
import numpy as np
dataset2 = pd.read_csv("merged_congestion_dummy.csv")
dataset2.shape

In [ ]:
from sklearn.preprocessing import StandardScaler
dataset2[['tti','speed','precipitation','tempurature']] = StandardScaler().fit_transform(dataset2[['tti','speed','precipitation','tempurature']])




In [ ]:
import numpy as np
y = np.array(dataset2["tti"].values)
X = dataset2.loc[:, "speed"::]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10 )
X_train, X_validate, y_train,y_validate = train_test_split(X_train, y_train, test_size=0.2, random_state=10 )
print("length of training %s || length of test %s"%(len(X_train),len(X_test)))

### Random search cross validation for the best_param for the congestion SVR

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint 
from scipy.stats import uniform as sp_uniform
from sklearn.svm import SVR

# model = SVR(kernel='rbf', gamma='auto_deprecated', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=800, verbose=False, max_iter=1000)
model = SVR()
param_dist = {
    'degree': sp_randint(1,10),
    'C' : sp_uniform(0,2),
    'epsilon':sp_uniform(0.05,0.15),
   
    
}
random_search = RandomizedSearchCV(
    model,
    param_distributions = param_dist,
    cv           = 5,
    n_iter       = 50,  
    random_state = 10 
    
)

### ==> computational Issue (even with the colab and HPC)

### ==> Manually tune the parameters

In [ ]:
from sklearn.svm import SVR
def training(C,epsilon):
    print("C = %s epsilon = %s"%(C,epsilon))
    
    model = SVR(degree = 1,C = C,epsilon = epsilon, shrinking=True, cache_size=1200, verbose=False, max_iter=3000)
    model.fit(X_train,y_train)
    y_predicted = model.predict(X_train)
    training_mse = mean_squared_error(y_predicted,y_train)
    print("The training_MSE is: %s"%training_mse)
    y_validate_predicted = model.predict(X_validate)
    validating_mse = mean_squared_error(y_validate,y_validate_predicted)
    print('The validating MSE is:%s '%validating_mse)


### tried the following parameter combination [on colab]
#### (C = 0.8, epsilon = 0.8) == > with max_iter = 3000, non convergence happening
#### (C= 1, epsilon = 1.0) == >The training_MSE is: 3.12, The validating MSE is:3.42
#### (C = 1.2, epsilon = 1.5) == > The training_MSE is: 3.35, The validating MSE is:3.62

### ==> choosing C = 0.8, epsilon = 0.1

In [10]:
from sklearn.svm import SVR
model = SVR(degree = 1,C = 0.8,epsilon = 0.1, shrinking=True, cache_size=1200, verbose=False, max_iter=1000)
model.fit(X_train,y_train)


/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:218: ConvergenceWarning: Solver terminated early (max_iter=1000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


SVR(C=0.8, cache_size=1200, coef0=0.0, degree=1, epsilon=0.1, gamma='auto',
  kernel='rbf', max_iter=1000, shrinking=True, tol=0.001, verbose=False)

In [11]:
from sklearn.metrics import mean_squared_error
y_predicted = model.predict(X_train)



In [12]:
training_mse = mean_squared_error(y_predicted,y_train)
print("The training_MSE is: %s"%training_mse)

The training_MSE is: 3.1181408032465905


In [13]:
y_validate_predicted = model.predict(X_validate)


In [14]:
validating_mse = mean_squared_error(y_validate,y_validate_predicted)
print('The validating MSE is:%s '%validating_mse)

The validating MSE is:3.416934061733038 
